# __Step 3.3: Cluster analysis__

The goals for step 3.3 are to:
- Identify "enriched" vocab words in each cluster
- Determine relations between clusters
- Assess the number of citations in each cluster chronologically

## ___Set up___

### Module import

In [1]:
import sys, os, pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing as mp
from pathlib import Path
from tqdm import tqdm
from scipy.stats import ks_2samp
from sklearn.cluster import SpectralClustering, KMeans

### Key variables

In [2]:
# Reproducibility
seed = 20220609

# Setting working directory
proj_dir   = Path.home() / "projects/plant_sci_hist"
work_dir   = proj_dir / "3_key_term_temporal/3_3_cluster_analysis"
work_dir.mkdir(parents=True, exist_ok=True)

os.chdir(work_dir)

# plant science corpus
dir25       = proj_dir / "2_text_classify/2_5_predict_pubmed"
corpus_file = dir25 / "corpus_plant_421658.tsv.gz"

# qualified feature names
dir31          = proj_dir / "3_key_term_temporal/3_1_pubmed_vocab"
X_vec_file     = dir31 / "tfidf_sparse_matrix_4542"
feat_name_file = dir31 / "tfidf_feat_name_and_sum_4542"

# fitted clustering objs
dir32            = proj_dir / "3_key_term_temporal/3_2_tf_idf_clustering"
clus_kmeans_file = dir32 / 'clus_kmeans'
cluster_num      = 500


## ___Get clustering results___

### KMeans

In [3]:
# Load kmean fit in case the session died.
with open(clus_kmeans_file, "rb") as f:
  clus_kmeans = pickle.load(f)

In [4]:
labels_kmeans = clus_kmeans.labels_
type(labels_kmeans), labels_kmeans.shape, labels_kmeans[:20]

(numpy.ndarray,
 (421658,),
 array([345, 310, 329,  66,   8, 299, 443, 116, 297, 297, 325, 121, 499,
        310, 162, 125, 116,  54,  67, 310], dtype=int32))

In [5]:
# {cluster_number: [indices of docs in the cluster]}
dict_kmeans = {}
for i in range(len(labels_kmeans)):
  label = labels_kmeans[i]
  if label not in dict_kmeans:
    dict_kmeans[label] = [i]
  else:
    dict_kmeans[label].append(i)


In [6]:
dict_kmeans_size = {}
for i in dict_kmeans:
  dict_kmeans_size[i] = len(dict_kmeans[i])
dict_kmeans_size_df = pd.DataFrame(list(dict_kmeans_size.items()), 
                                   columns=['cluster', 'size'])

In [7]:
dict_kmeans_size_df.describe()

,cluster,size
count,500.000000,500.000000
mean,249.500000,843.316000
std,144.481833,633.347277
min,0.000000,74.000000
25%,124.750000,424.750000
50%,249.500000,697.000000
75%,374.250000,1118.750000
max,499.000000,7915.000000


## ___"Enriched" vocab words___

- Am using Tf-idf values, so not the typically enrichment we are talking about here. More like, for a term, in a cluster X, the Tf-idf distribution is significantly differnt from Tf-idf values of the rest.
- So need to get a Tf-idf matrix for each cluster and anotehr for the rest.
- Do KS test, but instead of using all values, use max 10,000 values, if subsampling is needed, it is done 10 times. And the median p-value for KS test is used as the test stat.

### Read Tf-idf matrix and feat names

In [8]:
# Load sparse matrix from a pickle
with open(X_vec_file, 'rb') as f:
  X_vec = pickle.load(f)

# Load feature names and tf-idf sum
feat_sum = pd.read_csv(feat_name_file, sep='\t')

X_vec.shape, feat_sum.shape

((421658, 4542), (4542, 3))

### Functions

#### Get Tf-idx submatrix 

In [26]:
def get_submatrices(X_vec, labels, target_label):

  target_list  = []
  nontar_list  = []

  # Populate target and non-target lists with indices
  for i in range(len(labels_kmeans)):
    label = labels[i]
    if label == target_label:
      target_list.append(i)
    else:
      nontar_list.append(i)

  # convert to numpy array
  target_array = np.array(target_list)
  nontar_array = np.array(nontar_list)

  # Get the sparse matrix columns based on indices
  X_vec_target = X_vec[target_array, :]
  X_vec_nontar = X_vec[nontar_array, :]
  #print(f"  target:{X_vec_target.shape}, non-target:{X_vec_nontar.shape}")

  return X_vec_target, X_vec_nontar

#### For conducting ks_test

In [24]:
def ks_test(label):
 
  X_vec_target, X_vec_nontar = get_submatrices(X_vec, labels_kmeans, label)
  num_feat = X_vec_target.shape[1]

  dict_results = {} # {feat_index:[effect size, stat, pval]}
  for feat_index in range(num_feat):
    target_array = X_vec_target[:, feat_index].toarray().flatten()
    nontar_array = X_vec_nontar[:, feat_index].toarray().flatten()

    target_median = np.median(target_array)
    nontar_median = np.median(nontar_array)
    effect_size   = target_median-nontar_median

    if target_median > nontar_median:
      result = ks_2samp(target_array, nontar_array)
      dict_results[feat_index] = [effect_size, result.statistic, result.pvalue]

  return dict_results

### Get test stat for kmean clsuters through parallization

- https://superfastpython.com/multiprocessing-pool-apply/
- https://clay-atlas.com/us/blog/2021/08/02/python-en-use-multi-processing-pool-progress-bar/
- https://bentyeh.github.io/blog/20190722_Python-multiprocessing-progress.html
- https://inside-machinelearning.com/en/parallelization-in-python-getting-the-most-out-of-your-cpu/
- https://stackoverflow.com/questions/5666576/show-the-progress-of-a-python-multiprocessing-pool-imap-unordered-call

Was using `pool.apply` and it did not parallize. Then switch to `pool.imap`. I probably setup apply wrong anyway.

```Python
with mp.Pool(mp.cpu_count()) as pool:
  for label in range(cluster_num):
    dict_results = pool.apply(ks_test, args=(label,))
    dict_results_list.append(dict_results)
```

If I run as one process, each cluster is ~2min, so 500 cluster is 1000 min which is ~16-17 hours. 
- With the code below that presumably used 16 parallel processes, this should be done in 1 hours. But it went on for six hours. 
- Do this in HPC instead and break down the process into 20 jobs each with 25 clusters.

In [ ]:
# Go through different clusters

#dict_results_list = []
#with mp.Pool(mp.cpu_count()) as pool:
#  for dict_results in tqdm(pool.imap(ks_test, range(cluster_num)), 
#                           total=cluster_num):
#    dict_results_list.append(dict_results)

In [ ]:
# Dump the results as a pickle
#with open(work_dir / "dict_results_list_kmeans", 'wb') as f:
#  pickle.dump(dict_results_list, f)

### Consolidate test stat objects

The 500 clusters were processed in 20 jobs. Each lead to a dict_result_list that needs to be combined.

In [32]:
dict_results_list_all = []
for label in range(0, 500, 25):
  dict_results_list_file = work_dir / \
                              f'dict_results_list_kmeans_C{label}-{label+25}'
  with open(dict_results_list_file, "rb") as f:
    dict_results_list = pickle.load(f)
    print(label, len(dict_results_list))
    dict_results_list_all.extend(dict_results_list)

0 26
25 26
50 26
75 26
100 26
125 26
150 26
175 26
200 26
225 26
250 26
275 26
300 26
325 26
350 26
375 26
400 26
425 26
450 26
475 26


## ___FOR LATER___

### Read corpus file with date and journal info

In [ ]:
corpus_df = pd.read_csv(corpus_file, sep='\t', compression='gzip')
corpus_df.shape